In [3]:
def delete_folder(directory_dir, size= batch_rand * factor_no):
    folder_list = os.listdir(directory_dir)
    folder_dir = [os.path.join(directory_dir,i) for i in folder_list if len(os.listdir(os.path.join(directory_dir,i))) <= size ]
    for folder in folder_dir:
        print('deleting ' + folder + ' now...')
        shutil.rmtree(folder)
    print('completed!')


def cal_weight(class_name_list,IN_DIR):
    amounts_of_class_dict = {}
    mx = 0
    tot = 0
    for class_name in class_name_list:
        class_dir = IN_DIR + os.sep + class_name
        file_list = os.listdir(class_dir)
        amounts_of_class_dict[class_name] = len(file_list)
        if mx < len(file_list):
            mx = len(file_list)
    class_weights = {}
    count = 0
    for class_name in class_name_list:
        class_weights[count] = round(float(math.pow(amounts_of_class_dict[class_name]/mx, -1)),2)
        # weight = 1/(data nubmer/ MAX data number)
        count += 1
    return class_weights


def batch_iter(data, labels, batch_size, shuffle=True):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1

    def data_generator():
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]

            else:
                shuffled_data = data
                shuffled_labels = labels


            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                X = shuffled_data[start_index: end_index]
                y = shuffled_labels[start_index: end_index]
                yield X, y

    return num_batches_per_epoch, data_generator()


# Grad-CAM visualisation
# Grad-CAM for batched images
def grad_cam_batch(Input_model, Images, Classes, Layer_name, NULL_list, X_size, Y_size):
    if Y_size == 1:
        Loss = tf.gather_nd(Input_model.output, np.dstack([range(Images.shape[0]), Classes])[0])
        Layer_output = Input_model.get_layer(Layer_name).output
        Grads = K.gradients(Loss, Layer_output)[0]
        gradient_fnc = K.function([Input_model.input, tf.constant([0])], [Layer_output, Grads])

        conv_output, Grads_val = gradient_fnc([Images, tf.constant([0])])    
        weights = np.mean(Grads_val, axis=2)
        cams = np.einsum('ijk,ik->ij', conv_output.transpose(0,2,1), weights)

        # Process CAMs
        new_cams = np.zeros(X_size)
        for i in range(Images.shape[0]):
            cams[i] = np.maximum(cams[i], 0.)
            if np.max(cams[i]) != 0.:
                if math.isfinite(cams[i][0]):
                    new_cams += cams[i] / np.max(cams[i])
        return new_cams

    else:
        Loss = tf.gather_nd(Input_model.output, np.dstack([range(Images.shape[0]), Classes])[0])
        Layer_output = Input_model.get_layer(Layer_name).output
        Grads = K.gradients(Loss, Layer_output)[0]
        gradient_fnc = K.function([Input_model.input, tf.constant([0])], [Layer_output, Grads])

        conv_output, Grads_val = gradient_fnc([Images, tf.constant([0])])    
        weights = np.mean(Grads_val, axis=(2, 3))
        cams = np.einsum('ijkl,il->ijk', conv_output.transpose(0,2,3,1), weights)
        for i in range(len(NULL_list)):
            cams[:,NULL_list[i][0],NULL_list[i][1]] = 0

        # Process CAMs
        new_cams = np.zeros((X_size, Y_size))
        for i in range(Images.shape[0]):
            cams[i] = np.maximum(cams[i], 0.)
            if np.max(cams[i]) != 0.:
                if math.isfinite(cams[i][0][0]):
                    new_cams += cams[i] / np.max(cams[i])
        return new_cams

def grad_batch(Input_model, Images, Classes, Layer_name, NULL_list, X_size, Y_size):
    if Y_size == 1:
        Loss = tf.gather_nd(Input_model.output, np.dstack([range(Images.shape[0]), Classes])[0])
        Grads = K.gradients(Loss, Input_model.input)[0]
        gradient_fnc = K.function([Input_model.input], [Grads])
        Grads_val = gradient_fnc([Images])[0]
        new_cams = np.zeros(X_size)
        for i in range(Images.shape[0]):
            new_cams +=  Grads_val[i][0]/(np.abs(Grads_val[i][0]).max())
        return new_cams

    else:
        Loss = tf.gather_nd(Input_model.output, np.dstack([range(Images.shape[0]), Classes])[0])
        Grads = K.gradients(Loss, Input_model.input)[0]
        gradient_fnc = K.function([Input_model.input], [Grads])
        Grads_val = gradient_fnc([Images])[0]
        new_cams = np.zeros((X_size, Y_size))
        for i in range(Images.shape[0]):
            new_cams +=  Grads_val[i][0]/(np.abs(Grads_val[i][0]).max())
        return new_cams

def Grad_imaging(cam):
    jetcam = cv2.applyColorMap(np.uint8(256 * cam), cv2.COLORMAP_JET)
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)
    return jetcam

def Matthew(TP, TN, FP, FN):
    return ((TP*TN)-(FP*FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


def Grad_Cam_imaging(cam):
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)
    return jetcam

print("Functions were loaded.")




Functions were loaded.


In [ ]:
# removing very rare classes (less than or equal to 'factor_no' samples per class)

if imaging_pathway == 2 or imaging_pathway == 4:
    delete_folder(directory_dir = data_dir, size= batch_rand * factor_no)

n_categories = len(os.listdir(data_dir))


In [ ]:
# Downloading KEGG pathway annonation

if not os.path.exists(home_dir + '/KEGG'):
    os.mkdir(home_dir + '/KEGG')
if not os.path.exists(home_dir + '/info'):
    os.mkdir(home_dir + '/info')

url_anno = 'http://rest.kegg.jp/link/hsa/pathway'
url_gene = 'http://rest.kegg.jp/list/hsa'
url_path = 'http://rest.kegg.jp/list/pathway/hsa'
url_ncbi = 'http://rest.kegg.jp/conv/hsa/ncbi-geneid'

try:
    r_a = requests.get(url_anno)
    r_g = requests.get(url_gene)
    r_p = requests.get(url_path)
    r_n = requests.get(url_ncbi)

    with open(home_dir + '/KEGG/link.tsv', mode='w') as f_a:
        f_a.write(r_a.text)
    with open(home_dir + '/KEGG/genes.tsv', mode='w') as f_g:
        f_g.write(r_g.text)
    with open(home_dir + '/KEGG/pathway.tsv', mode='w') as f_p:
        f_p.write(r_p.text)
    with open(home_dir + '/KEGG/ncbi_geneid.tsv', mode='w') as f_n:
        f_n.write(r_n.text)

except requests.exceptions.RequestException as err:
    print(err)


In [ ]:
# Feature selected, top-10, 309 genes

###### Making template from KEGG pathway information files
# only primary tumor samples
# TCGA project: 9,911 samples
# major pathogolic types from GDC database: 11,997 samples

# 60483 features
# using only 308 genes selected by Pagging method

# making 308 x 1 pxls images with 32 patterns

if not os.path.exists(home_dir + '/array_select_309'):
    os.mkdir(home_dir + '/array_select_309/')
if not os.path.exists(home_dir + '/image_select_309'):
    os.mkdir(home_dir + '/image_select_309')
if not os.path.exists(home_dir + '/select_patho_309'):
    os.mkdir(home_dir + '/select_patho_309')

X_size_ = X_size
Y_size_ = Y_size
X_size = 309
Y_size = 1

df_genes = pd.read_table(home_dir + '/KEGG/genes.tsv', header=None)
df_link = pd.read_table(home_dir + '/KEGG/link.tsv', header=None)
df_pathway = pd.read_table(home_dir + '/KEGG/pathway.tsv', header=None)
df_ncbi_geneid = pd.read_table(home_dir + '/KEGG/ncbi_geneid.tsv', header=None)
df_ENSG = pd.read_table(home_dir + '/info/ENSG_to_NCBI.tsv', header=0, index_col=0)
df_FPKM = pd.read_table(home_dir + '/info/FPKM_test.tsv', header=None)
df_select = pd.read_table(home_dir + '/info/309_genes.tsv', header=None)


df_FPKM.columns = ['ENSG', 'FPKM']
df_FPKM["NCBI"] = "NA"
df_FPKM["Type"] = "NA"
df_FPKM["ENSG_no_ver"] = "NA"
df_FPKM["FPKM_log2"] = np.log2(df_FPKM["FPKM"]+1)
df_FPKM["Gene_name"] = "NA"
df_FPKM = df_FPKM[df_FPKM['ENSG'].str.split('.', expand=True)[0].isin(list(df_select[0]))]

for i in range(len(df_FPKM["ENSG"])):
    if i % 500 ==0:
        print(i)
    df_FPKM.iloc[i,4] = df_FPKM.iloc[i,0].split(".")[0]
    try:
        tmp = df_ENSG.loc[df_FPKM.iloc[i,4]]
    except:
        continue
    if tmp["entrezgene_id"].size == 1:
        df_FPKM.iloc[i,2] = tmp.iloc[2]
        df_FPKM.iloc[i,3] = tmp.iloc[1]
        df_FPKM.iloc[i,6] = tmp.iloc[0]
    else:
        for j in range(tmp["gene_biotype"].size):
            tmp2 = pd.Series([df_FPKM.iloc[i,0], df_FPKM.iloc[i,1], tmp.iloc[j,2], tmp.iloc[j,1], df_FPKM.iloc[i,4], df_FPKM.iloc[i,5], tmp.iloc[j,0]], index=['ENSG','FPKM','NCBI','Type','ENSG_no_ver','FPKM_log2','Gene_name'])
            df_FPKM = df_FPKM.append([tmp2])

df_ncbi_geneid.columns = ['KEGG', 'NCBI']
for i in range(len(df_ncbi_geneid)):
    if i % 1000 == 0:
        print(i)
    df_ncbi_geneid.iloc[i,0] = df_ncbi_geneid.iloc[i,0].lstrip("hsa:")
    df_ncbi_geneid.iloc[i,1] = df_ncbi_geneid.iloc[i,1].lstrip("ncbi-geneid:")

df_ncbi_geneid_ = df_ncbi_geneid.set_index('KEGG')
df_link.columns = ['KEGG_PATH', 'KEGG_GENE']
df_link["NCBI"] = "NA"

for i in range(len(df_link)):
    if i % 1000 == 0:
        print(i)
    df_link.iloc[i,0] = df_link.iloc[i,0].lstrip("path:hsa")
    df_link.iloc[i,1] = df_link.iloc[i,1].lstrip("hsa:")
    try:
        df_link.iloc[i,2] = df_ncbi_geneid_.loc[str(int(df_link.iloc[i,1]))].iloc[0]
    except:
        df_link.iloc[i,2] = "NA"

df_FPKM_ = df_FPKM.copy()
df_link_ = df_link.copy()
df_link_["FPKM_log2"] = 0
df_link_["ENSG_no_var"] = "NA"
df_link_["Gene_name"] = "NA"
df_link_["Type"] = "NA"
df_link_["index"] = 0
df_FPKM_["Check"] = 0
df_FPKM_ = df_FPKM_.set_index('NCBI')

for i in range(len(df_link_)):
    if i % 1000 == 0:
        print(i)
    try:
        j = float(df_link_.iloc[i,2])
        tmp = df_FPKM_.loc[j]
        df_link_.iloc[i,3] = tmp.iloc[4]
        df_link_.iloc[i,4] = tmp.iloc[3]
        df_link_.iloc[i,5] = tmp.iloc[5]
        df_link_.iloc[i,6] = tmp.iloc[2]
        df_link_.iloc[i,7] = tmp.iloc[1]
        df_FPKM_.loc[j,df_FPKM_.columns[6]] = 1

    except:
        continue

df_FPKM__ = df_FPKM_[df_FPKM_["Check"] == 0].drop_duplicates(subset='ENSG', keep='last')
df_FPKM___ = df_FPKM__.reset_index()

tmp = pd.DataFrame(df_FPKM___.iloc[:,7])
tmp["tmp1"] = df_FPKM___.iloc[:,7]
tmp["tmp2"] = df_FPKM___.iloc[:,0]
tmp["tmp3"] = df_FPKM___.iloc[:,5]
tmp["tmp4"] = df_FPKM___.iloc[:,4]
tmp["tmp5"] = df_FPKM___.iloc[:,6]
tmp["tmp6"] = df_FPKM___.iloc[:,3]
tmp["tmp7"] = df_FPKM___.iloc[:,2]

tmp.columns = ['KEGG_GENE', 'KEGG_PATH', 'NCBI', 'FPKM_log2', 'ENSG_no_var', 'Gene_name', 'Type', 'index']
tmp = tmp.sort_values(by=["Type"], ascending=True)

df_template = pd.concat([df_link_, tmp])
df_template = df_template.drop('FPKM_log2', axis = 1)
df_template = df_template[df_template['ENSG_no_var'] != "NA"]
df_template['KEGG_PATH'] = df_template['KEGG_PATH'].astype(int)

df_template.to_csv(home_dir + '/info/template_308_genes.tsv', index=False, sep='\t')

tmp1 = df_template[df_template['KEGG_PATH'] != 0]
tmp1 = tmp1.drop_duplicates(subset='ENSG_no_var',keep="first")

data_No = tmp1['ENSG_no_var'].size
unique_gene_No = tmp1['ENSG_no_var'].unique().size
temp_size = df_FPKM["FPKM"].max()

for j in range(batch_rand):
    print(j)
    img_name = home_dir + '/array_select_309/img_' + str(j) + '.tsv'
    df_template = tmp1.take(np.random.permutation(len(tmp1)))
    img = temp_size * np.ones((X_size, Y_size))
    for i in range(data_No):
        img[i, 0] = df_template.iloc[i,6]
    pd.DataFrame(img).astype('int32').to_csv(img_name, index=False, sep='\t')

X_size = X_size_
Y_size = Y_size_







In [ ]:
# TOP v6, 729 genes

###### Making template from KEGG pathway information files
# only primary tumor samples
# TCGA project: 9,911 samples
# major pathogolic types from GDC database: 11,997 samples

# 60483 features
# using only 308 genes selected by Pagging method

# making 308 x 1 pxls images with 32 patterns

if not os.path.exists(home_dir + '/array_top_v6'):
    os.mkdir(home_dir + '/array_top_v6/')
if not os.path.exists(home_dir + '/image_top_v6'):
    os.mkdir(home_dir + '/image_top_v6')
if not os.path.exists(home_dir + '/top_v6'):
    os.mkdir(home_dir + '/top_v6')

X_size_ = X_size
Y_size_ = Y_size
X_size = 729
Y_size = 1

df_genes = pd.read_table(home_dir + '/KEGG/genes.tsv', header=None)
df_link = pd.read_table(home_dir + '/KEGG/link.tsv', header=None)
df_pathway = pd.read_table(home_dir + '/KEGG/pathway.tsv', header=None)
df_ncbi_geneid = pd.read_table(home_dir + '/KEGG/ncbi_geneid.tsv', header=None)
df_ENSG = pd.read_table(home_dir + '/info/ENSG_to_NCBI.tsv', header=0, index_col=0)
df_FPKM = pd.read_table(home_dir + '/info/FPKM_test.tsv', header=None)
df_select = pd.read_table(home_dir + '/info/top_v6_729_genes.tsv', header=None)


df_FPKM.columns = ['ENSG', 'FPKM']
df_FPKM["NCBI"] = "NA"
df_FPKM["Type"] = "NA"
df_FPKM["ENSG_no_ver"] = "NA"
df_FPKM["FPKM_log2"] = np.log2(df_FPKM["FPKM"]+1)
df_FPKM["Gene_name"] = "NA"
df_FPKM = df_FPKM[df_FPKM['ENSG'].str.split('.', expand=True)[0].isin(list(df_select[0]))]

for i in range(len(df_FPKM["ENSG"])):
    if i % 500 ==0:
        print(i)
    df_FPKM.iloc[i,4] = df_FPKM.iloc[i,0].split(".")[0]
    try:
        tmp = df_ENSG.loc[df_FPKM.iloc[i,4]]
    except:
        continue
    if tmp["entrezgene_id"].size == 1:
        df_FPKM.iloc[i,2] = tmp.iloc[2]
        df_FPKM.iloc[i,3] = tmp.iloc[1]
        df_FPKM.iloc[i,6] = tmp.iloc[0]
    else:
        for j in range(tmp["gene_biotype"].size):
            tmp2 = pd.Series([df_FPKM.iloc[i,0], df_FPKM.iloc[i,1], tmp.iloc[j,2], tmp.iloc[j,1], df_FPKM.iloc[i,4], df_FPKM.iloc[i,5], tmp.iloc[j,0]], index=['ENSG','FPKM','NCBI','Type','ENSG_no_ver','FPKM_log2','Gene_name'])
            df_FPKM = df_FPKM.append([tmp2])

df_ncbi_geneid.columns = ['KEGG', 'NCBI']
for i in range(len(df_ncbi_geneid)):
    if i % 1000 == 0:
        print(i)
    df_ncbi_geneid.iloc[i,0] = df_ncbi_geneid.iloc[i,0].lstrip("hsa:")
    df_ncbi_geneid.iloc[i,1] = df_ncbi_geneid.iloc[i,1].lstrip("ncbi-geneid:")

df_ncbi_geneid_ = df_ncbi_geneid.set_index('KEGG')
df_link.columns = ['KEGG_PATH', 'KEGG_GENE']
df_link["NCBI"] = "NA"

for i in range(len(df_link)):
    if i % 1000 == 0:
        print(i)
    df_link.iloc[i,0] = df_link.iloc[i,0].lstrip("path:hsa")
    df_link.iloc[i,1] = df_link.iloc[i,1].lstrip("hsa:")
    try:
        df_link.iloc[i,2] = df_ncbi_geneid_.loc[str(int(df_link.iloc[i,1]))].iloc[0]
    except:
        df_link.iloc[i,2] = "NA"

df_FPKM_ = df_FPKM.copy()
df_link_ = df_link.copy()
df_link_["FPKM_log2"] = 0
df_link_["ENSG_no_var"] = "NA"
df_link_["Gene_name"] = "NA"
df_link_["Type"] = "NA"
df_link_["index"] = 0
df_FPKM_["Check"] = 0
df_FPKM_ = df_FPKM_.set_index('NCBI')

for i in range(len(df_link_)):
    if i % 1000 == 0:
        print(i)
    try:
        j = float(df_link_.iloc[i,2])
        tmp = df_FPKM_.loc[j]
        df_link_.iloc[i,3] = tmp.iloc[4]
        df_link_.iloc[i,4] = tmp.iloc[3]
        df_link_.iloc[i,5] = tmp.iloc[5]
        df_link_.iloc[i,6] = tmp.iloc[2]
        df_link_.iloc[i,7] = tmp.iloc[1]
        df_FPKM_.loc[j,df_FPKM_.columns[6]] = 1

    except:
        continue

df_FPKM__ = df_FPKM_[df_FPKM_["Check"] == 0].drop_duplicates(subset='ENSG', keep='last')
df_FPKM___ = df_FPKM__.reset_index()

tmp = pd.DataFrame(df_FPKM___.iloc[:,7])
tmp["tmp1"] = df_FPKM___.iloc[:,7]
tmp["tmp2"] = df_FPKM___.iloc[:,0]
tmp["tmp3"] = df_FPKM___.iloc[:,5]
tmp["tmp4"] = df_FPKM___.iloc[:,4]
tmp["tmp5"] = df_FPKM___.iloc[:,6]
tmp["tmp6"] = df_FPKM___.iloc[:,3]
tmp["tmp7"] = df_FPKM___.iloc[:,2]

tmp.columns = ['KEGG_GENE', 'KEGG_PATH', 'NCBI', 'FPKM_log2', 'ENSG_no_var', 'Gene_name', 'Type', 'index']
tmp = tmp.sort_values(by=["Type"], ascending=True)

df_template = pd.concat([df_link_, tmp])
df_template = df_template.drop('FPKM_log2', axis = 1)
df_template = df_template[df_template['ENSG_no_var'] != "NA"]
df_template['KEGG_PATH'] = df_template['KEGG_PATH'].astype(int)

df_template.to_csv(home_dir + '/info/template_top_v6.tsv', index=False, sep='\t')

tmp1 = df_template[df_template['KEGG_PATH'] != -1]
tmp1 = tmp1.drop_duplicates(subset='ENSG_no_var',keep="first")

data_No = tmp1['ENSG_no_var'].size
unique_gene_No = tmp1['ENSG_no_var'].unique().size
temp_size = df_FPKM["FPKM"].max()

for j in range(batch_rand):
    print(j)
    img_name = home_dir + '/array_top_v6/img_' + str(j) + '.tsv'
    df_template = tmp1.take(np.random.permutation(len(tmp1)))
    img = temp_size * np.ones((X_size, Y_size))
    for i in range(data_No):
        img[i, 0] = df_template.iloc[i,6]
    pd.DataFrame(img).astype('int32').to_csv(img_name, index=False, sep='\t')

X_size = X_size_
Y_size = Y_size_








In [ ]:
##### learning with 1D-CNNs
# 32 CNNs and pagging

if not os.path.exists(result_dir + "/test_index"):
    os.mkdir(result_dir + "/test_index")
if not os.path.exists(result_dir + "/weight"):
    os.mkdir(result_dir + "/weight")
if not os.path.exists(result_dir + "/conf"):
    os.mkdir(result_dir + "/conf")
if not os.path.exists(result_dir + "/predict"):
    os.mkdir(result_dir + "/predict")
if not os.path.exists(result_dir + "/acc"):
    os.mkdir(result_dir + "/acc")
if not os.path.exists(result_dir + "/loss"):
    os.mkdir(result_dir + "/loss")

batch_rand=32
#NORMAL = "_normal"
NORMAL = ""
#n_categories = 46

accuracy_table = []

trainLabel = pd.read_table(data_dir + '_result' + "/npy/label" + NORMAL + ".tsv", header=0)
label_List = np.asarray(trainLabel).reshape(-1,)
le = LabelEncoder()
le = le.fit(label_List)
label_List = le.transform(label_List)
tmp = trainLabel["0"].unique()
tmp.sort()
tmp1 = pd.DataFrame(tmp)
cal_weights = cal_weight(tmp, data_dir)

for bat in range(0, batch_rand):
    kf = StratifiedKFold(n_splits=n_Splits, shuffle=True, random_state = seed1+bat)

    image_list = np.load(data_dir + '_result' + "/npy/image_" + str(bat) + NORMAL + ".npz")['arr_0'].astype(np.float32)
    image_list = np.reshape(image_list, (-1, 1, X_size))


    gc.collect()

    # 5-fold cross validation
    # train:valid:test = 60:20:20
    # same separation in any batch
    cv_batch = -1
    for train_valid_index, test_index in kf.split(image_list, label_List):
        cv_batch += 1
        if(cv_batch != 5):
            label_list = np_utils.to_categorical(label_List, n_categories)
            np.save(result_dir + "/test_index/cv_batch_" + str(bat) + "_"  + str(cv_batch) + "_index"+ NORMAL + ".npy", test_index)
            X_train, X_valid, y_train, y_valid = train_test_split(image_list[train_valid_index], label_list[train_valid_index], random_state=seed2+bat, test_size=split, stratify=label_list[train_valid_index])
            X_test = image_list[test_index]
            y_test = label_list[test_index]

            train_steps, train_batches = batch_iter(X_train, y_train, batch_size)
            valid_steps, valid_batches = batch_iter(X_valid, y_valid, batch_size)

            gc.collect()

            # training
            inputs = Input(shape=(1, X_size))
            x = Conv1D(32, 32, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(inputs)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)
            #x = Conv1D(64, 16, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)
            x = Conv1D(32, 128, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)
            x = Conv1D(32, 256, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)
            #x = Conv1D(64, 4, strides=4, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)
            #x = Conv1D(128, 4, strides=4, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)


            #x = Conv1D(512, 8, strides=8, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)







            #x = Conv1D(256, 128, strides=128, padding='same', data_format="channels_first", name='block1_conv2', kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)
            #x = MaxPooling1D(pool_size=2, strides=None, padding='same', data_format="channels_first")(x)

            #x = Conv1D(32, 256, strides=1, padding='same', data_format="channels_first", name='block1_conv3', kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=True)
            x = Flatten(name='flatten')(x)
            x = Dense(512, activation='relu', name='fc1')(x)
            x = Dropout(0.2, name='dropout1')(x,training=True)


            predictions = Dense(n_categories, activation='softmax', name='predictions')(x)
            VGG_model = Model(inputs=inputs, outputs=predictions)
            VGG_model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.SGD(lr=learning, momentum=0.9, decay=1e-4, nesterov=True),
                      metrics=["acc", Precision(), Recall()])
            if bat == 0 and cv_batch == 0:
                VGG_model.save_weights(result_dir + "/weight/initial_weights"+ NORMAL + ".h5")
            else:
                VGG_model.load_weights(result_dir + "/weight/initial_weights"+ NORMAL + ".h5")

            gc.collect()
            early_stopping = EarlyStopping(monitor='val_loss', patience=patience , verbose=0)
            checkpointer = ModelCheckpoint(monitor='val_loss', filepath = result_dir + "/weight/rand_" + str(bat) + "_" + str(cv_batch) + "_model"+ NORMAL + ".h5", save_weights_only=True, verbose=0, mode='auto', save_best_only=True)
            reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1, min_delta=0.001, min_lr=0.000005)
            print("\rtraining: " + str(bat) + "_" + str(cv_batch), end='')

            history = VGG_model.fit_generator(train_batches, train_steps,
                                epochs=epochs,
                                class_weight=cal_weights,
                                validation_data=valid_batches,
                                validation_steps=valid_steps,
                                verbose=1, 
                                callbacks=[reduce_lr, early_stopping, checkpointer])

            del VGG_model
            del x, inputs, predictions

            inputs = Input(shape=(1, X_size))
            x = Conv1D(32, 32, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(inputs)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            #x = Conv1D(64, 16, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=False)
            x = Conv1D(32, 128, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            x = Conv1D(32, 256, strides=1, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            #x = Conv1D(64, 4, strides=4, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=False)
            #x = Conv1D(128, 4, strides=4, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=False)

            #x = Conv1D(512, 8, strides=8, padding='same', data_format="channels_first", kernel_initializer='he_normal')(x)
            #x = ReLU(max_value=None)(x)
            #x = BatchNormalization(axis=1)(x,training=False)

            x = Flatten(name='flatten')(x)
            x = Dense(512, activation='relu', name='fc1')(x)
            x = Dropout(0.2, name='dropout1')(x,training=False)


            predictions = Dense(n_categories, activation='softmax', name='predictions')(x)
            VGG_model = Model(inputs=inputs, outputs=predictions)

            VGG_model.load_weights(result_dir + "/weight/rand_" + str(bat) + "_" + str(cv_batch) + "_model"+ NORMAL + ".h5")
            y_pred = VGG_model.predict(X_test, batch_size=batch_size, verbose=0)
            tmp2 = pd.DataFrame(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
            pd.concat([tmp1, tmp2], axis=1).to_csv(result_dir + "/conf/rand_" + str(bat) + "_" + str(cv_batch)  + "_conftable"+ NORMAL + ".tsv", index=False, sep='\t')
            np.save(result_dir + "/predict/rand_" + str(bat) + "_" + str(cv_batch)  + "_predict"+ NORMAL + ".npy", y_pred)

            xlab = np.arange(0, len(history.history["acc"])) + 1

            # model evaluation
            acc_txt = str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
            f1_txt = str(f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average="macro"))
            save_txt = 'Batch ' + str(bat) + "_" + str(cv_batch)  + ', Test accuracy/F1 score : ' + acc_txt + ' / ' + f1_txt + NORMAL + "\n"
            f = open(acc_name, 'a')
            f.write(save_txt)
            f.close()
            accuracy_table.append([bat, cv_batch, accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)),f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average="macro")])
            pd.DataFrame(np.asarray(accuracy_table)).to_csv(acc_table, index=False, sep='\t')


            #plot accuracy
            plt.plot(range(1, len(history.history["acc"])+1), history.history["acc"], label = "Training acc" )
            plt.plot(range(1, len(history.history["val_acc"])+1), history.history["val_acc"], label = "Validation acc")
            plt.title("Training and Validation accuracy")
            plt.legend()
            plt.savefig(result_dir + "/acc/rand_" + str(bat) + "_" + str(cv_batch)  + "_acc"+ NORMAL + ".png")
            plt.clf()

            #plot loss
            plt.plot(range(1, len(history.history["loss"])+1), history.history["loss"],  label = "Training loss" )
            plt.plot(range(1, len(history.history["val_loss"])+1), history.history["val_loss"], label = "Validation loss")
            plt.title("Training and Validation loss")
            plt.legend()
            plt.savefig(result_dir + "/loss/rand_" + str(bat) + "_" + str(cv_batch)  + "_loss"+ NORMAL + ".png")
            plt.clf()

            K.clear_session()
            gc.collect()
            del VGG_model
            del x, inputs, predictions
            del history
            del early_stopping
            del checkpointer, reduce_lr
            del X_train, X_valid, X_test
            del train_steps, train_batches
            del valid_steps, valid_batches


# ensembling

print("ensembling")

predict_ensemble = np.zeros((len(label_list),n_categories))

for cv_batch in range(0, n_Splits):
    for bat in range(0, batch_rand):
        test_index = np.load(result_dir + "/test_index/cv_batch_" + str(bat) + "_"  + str(cv_batch) + "_index"+ NORMAL + ".npy")
        predict = np.load(result_dir + "/predict/rand_" + str(bat) + "_" + str(cv_batch) + "_predict"+ NORMAL + ".npy")
        for i in range(len(test_index)):
            predict_ensemble[test_index] += predict[i]
y_test = label_list
tmp2 = pd.DataFrame(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1)))
pd.concat([tmp1, tmp2], axis=1).to_csv(result_dir + "/conf/ensemble_conftable" + NORMAL + ".tsv", index=False, sep='\t')
np.save(result_dir + "/predict/ensemble_predict"+ NORMAL + ".npy", predict_ensemble)

acc_txt = str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1)))
f1_txt = str(f1_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1), average="macro"))
save_txt = 'Ensemble, Test accuracy/F1 score : ' + acc_txt + ' / ' + f1_txt+ NORMAL +  "\n"
f = open(acc_name, 'a')
f.write(save_txt)
f.close()




In [ ]:
##### learning with 1D-CNNs
# 32 CNNs and pagging
# small scale

batch_rand=32

if not os.path.exists(result_dir + "/test_index"):
    os.mkdir(result_dir + "/test_index")
if not os.path.exists(result_dir + "/weight"):
    os.mkdir(result_dir + "/weight")
if not os.path.exists(result_dir + "/conf"):
    os.mkdir(result_dir + "/conf")
if not os.path.exists(result_dir + "/predict"):
    os.mkdir(result_dir + "/predict")
if not os.path.exists(result_dir + "/acc"):
    os.mkdir(result_dir + "/acc")
if not os.path.exists(result_dir + "/loss"):
    os.mkdir(result_dir + "/loss")

#batch_rand=32
#NORMAL = "_normal"
NORMAL = ""
#n_categories = 33

accuracy_table = []

trainLabel = pd.read_table(data_dir + '_result' + "/npy/label" + NORMAL + ".tsv", header=0)
label_List = np.asarray(trainLabel).reshape(-1,)
le = LabelEncoder()
le = le.fit(label_List)
label_List = le.transform(label_List)
tmp = trainLabel["0"].unique()
tmp.sort()
tmp1 = pd.DataFrame(tmp)

for bat in range(0, batch_rand):
    kf = StratifiedKFold(n_splits=n_Splits, shuffle=True, random_state = seed1)

    image_list = np.load(data_dir + '_result' + "/npy/image_" + str(bat) + NORMAL + ".npz")['arr_0'].astype(np.float32)
    image_list = np.reshape(image_list, (-1, 1, X_size))


    gc.collect()

    # 5-fold cross validation
    # train:valid:test = 60:20:20
    # same separation in any batch
    cv_batch = -1
    for train_valid_index, test_index in kf.split(image_list, label_List):
        cv_batch += 1
        if(cv_batch != 5):

            label_list = np_utils.to_categorical(label_List, n_categories)
            np.save(result_dir + "/test_index/cv_batch_" + str(cv_batch) + "_index"+ NORMAL + ".npy", test_index)
            X_train, X_valid, y_train, y_valid = train_test_split(image_list[train_valid_index], label_list[train_valid_index], random_state=seed2+batch_rand, test_size=split, stratify=label_list[train_valid_index])
            X_test = image_list[test_index]
            y_test = label_list[test_index]

            cal_weights = cal_weight(tmp, data_dir)
            train_steps, train_batches = batch_iter(X_train, y_train, batch_size)
            valid_steps, valid_batches = batch_iter(X_valid, y_valid, batch_size)

            gc.collect()

            # training
            inputs = Input(shape=(1, X_size))
            x = Conv1D(32, 16, strides=1, padding='same', data_format="channels_first", name='block1_conv1', kernel_initializer='he_normal')(inputs)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)
            x = Conv1D(32, 64, strides=1, padding='same', data_format="channels_first", name='block1_conv2', kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)
            x = Conv1D(32, 128, strides=1, padding='same', data_format="channels_first", name='block1_conv3', kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=True)

            x = Flatten(name='flatten')(x)
            x = Dense(512, activation='relu', name='fc1')(x)
            x = Dropout(0.1, name='dropout1')(x,training=True)

            predictions = Dense(n_categories, activation='softmax', name='predictions')(x)
            VGG_model = Model(inputs=inputs, outputs=predictions)
            VGG_model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.SGD(lr=learning, momentum=0.9, decay=1e-4, nesterov=True),
                      #optimizer=tf.keras.optimizers.Adagrad(lr=learning, decay=1e-4),
                      #optimizer=tf.keras.optimizers.Nadam(lr=learning, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004),
                      metrics=["acc", Precision(), Recall()])
            if bat == 0 and cv_batch == 0:
                VGG_model.save_weights(result_dir + "/weight/initial_weights"+ NORMAL + ".h5")
            else:
                VGG_model.load_weights(result_dir + "/weight/initial_weights"+ NORMAL + ".h5")

            gc.collect()
            early_stopping = EarlyStopping(monitor='val_loss', patience=patience , verbose=0)
            checkpointer = ModelCheckpoint(monitor='val_loss', filepath = result_dir + "/weight/rand_" + str(bat) + "_" + str(cv_batch) + "_model"+ NORMAL + ".h5", save_weights_only=True, verbose=0, mode='min', save_best_only=True)
            reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1, min_delta=0.001, min_lr=0.00001)
            print("\rtraining: " + str(bat) + "_" + str(cv_batch), end='')

            history = VGG_model.fit_generator(train_batches, train_steps,
                                epochs=epochs,
                                class_weight=cal_weights,
                                validation_data=valid_batches,
                                validation_steps=valid_steps,
                                verbose=1, 
                                callbacks=[reduce_lr, early_stopping, checkpointer])

            del VGG_model
            del x, inputs, predictions

            inputs = Input(shape=(1, X_size))
            x = Conv1D(32, 16, strides=1, padding='same', data_format="channels_first", name='block1_conv1', kernel_initializer='he_normal')(inputs)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            x = Conv1D(32, 64, strides=1, padding='same', data_format="channels_first", name='block1_conv2', kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            x = Conv1D(32, 128, strides=1, padding='same', data_format="channels_first", name='block1_conv3', kernel_initializer='he_normal')(x)
            x = ReLU(max_value=None)(x)
            x = BatchNormalization(axis=1)(x,training=False)
            x = Flatten(name='flatten')(x)
            x = Dense(512, activation='relu', name='fc1')(x)
            x = Dropout(0.1, name='dropout1')(x,training=False)

            predictions = Dense(n_categories, activation='softmax', name='predictions')(x)
            VGG_model = Model(inputs=inputs, outputs=predictions)

            VGG_model.load_weights(result_dir + "/weight/rand_" + str(bat) + "_" + str(cv_batch) + "_model"+ NORMAL + ".h5")
            y_pred = VGG_model.predict(X_test, batch_size=batch_size, verbose=0)
            tmp2 = pd.DataFrame(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
            pd.concat([tmp1, tmp2], axis=1).to_csv(result_dir + "/conf/rand_" + str(bat) + "_" + str(cv_batch)  + "_conftable"+ NORMAL + ".tsv", index=False, sep='\t')
            np.save(result_dir + "/predict/rand_" + str(bat) + "_" + str(cv_batch)  + "_predict"+ NORMAL + ".npy", y_pred)

            xlab = np.arange(0, len(history.history["acc"])) + 1

            # model evaluation
            acc_txt = str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
            f1_txt = str(f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average="macro"))
            save_txt = 'Batch ' + str(bat) + "_" + str(cv_batch)  + ', Test accuracy/F1 score : ' + acc_txt + ' / ' + f1_txt + NORMAL + "\n"
            f = open(acc_name, 'a')
            f.write(save_txt)
            f.close()
            accuracy_table.append([bat, cv_batch, accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)),f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average="macro")])
            pd.DataFrame(np.asarray(accuracy_table)).to_csv(acc_table, index=False, sep='\t')


            #plot accuracy
            plt.plot(range(1, len(history.history["acc"])+1), history.history["acc"], label = "Training acc" )
            plt.plot(range(1, len(history.history["val_acc"])+1), history.history["val_acc"], label = "Validation acc")
            plt.title("Training and Validation accuracy")
            plt.legend()
            plt.savefig(result_dir + "/acc/rand_" + str(bat) + "_" + str(cv_batch)  + "_acc"+ NORMAL + ".png")
            plt.clf()

            #plot loss
            plt.plot(range(1, len(history.history["loss"])+1), history.history["loss"],  label = "Training loss" )
            plt.plot(range(1, len(history.history["val_loss"])+1), history.history["val_loss"], label = "Validation loss")
            plt.title("Training and Validation loss")
            plt.legend()
            plt.savefig(result_dir + "/loss/rand_" + str(bat) + "_" + str(cv_batch)  + "_loss"+ NORMAL + ".png")
            plt.clf()

            K.clear_session()
            gc.collect()
            del VGG_model
            del x, inputs, predictions
            del history
            del early_stopping
            del checkpointer, reduce_lr
            del cal_weights
            del X_train, X_valid, X_test
            del train_steps, train_batches
            del valid_steps, valid_batches


# ensembling

print("ensembling")

for cv_batch in range(0,n_Splits):
    test_index = np.load(result_dir + "/test_index/cv_batch_" + str(cv_batch) + "_index"+ NORMAL + ".npy")
    y_test = label_list[test_index]
    predict_ensemble = np.load(result_dir + "/predict/rand_" + "0" + "_" + str(cv_batch) + "_predict"+ NORMAL + ".npy")
    for bat in range(1, batch_rand):
        predict_ensemble += np.load(result_dir + "/predict/rand_" + str(bat) + "_" + str(cv_batch) + "_predict"+ NORMAL + ".npy")
    predict_ensemble /= batch_rand

    tmp2 = pd.DataFrame(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1)))
    pd.concat([tmp1, tmp2], axis=1).to_csv(result_dir + "/conf/ensemble_conftable" + "_" + str(cv_batch)+ NORMAL + ".tsv", index=False, sep='\t')
    np.save(result_dir + "/predict/ensemble_predict" + "_" + str(cv_batch)+ NORMAL + ".npy", predict_ensemble)

    acc_txt = str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1)))
    f1_txt = str(f1_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1), average="macro"))
    save_txt = 'Ensemble, Test accuracy/F1 score : ' + acc_txt + ' / ' + f1_txt+ NORMAL +  "\n"
    f = open(acc_name, 'a')
    f.write(save_txt)
    f.close()
    accuracy_table.append([999, cv_batch, accuracy_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1)),f1_score(np.argmax(y_test, axis=1), np.argmax(predict_ensemble, axis=1), average="macro")])
    pd.DataFrame(np.asarray(accuracy_table)).to_csv(acc_table, index=False, sep='\t')



In [ ]:
# Classification accuracy & confusion matrix

#ACCURACY = pd.read_table(home_dir + "/select_top1_patho_result/conf/rand_16_0_conftable.tsv", header=0, index_col=0)
#NORMAL = ""
#DIRECTORY = home_dir + "/select_patho_308_result/conf/ensemble_conftable_"

#NORMAL = "_normal"
NORMAL = ""

DIRECTORY = home_dir + "/unique_1D_patho_result/conf/ensemble_conftable"
ACCURACY = pd.read_table(DIRECTORY + ".tsv", header=0, index_col=0)

#ACCURACY = pd.read_table(DIRECTORY + "0"  + NORMAL + ".tsv", header=0, index_col=0)
#for i in range(1,5):
#    ACCURACY += pd.read_table(DIRECTORY + str(i)  + NORMAL +  ".tsv", header=0, index_col=0)

#cv_batch = 0
#bat = 0
#FILE = home_dir + "/KEGG_1D_TCGA_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#FILE = home_dir + "/unique_1D_KEGG_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#FILE = home_dir + "/unique_1D_patho_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#ACCURACY = pd.read_table(FILE, header=0, index_col=0)
#for cv_batch in range(1,5):
#    FILE = home_dir + "/KEGG_1D_TCGA_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#    FILE = home_dir + "/unique_1D_KEGG_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#    FILE = home_dir + "/unique_1D_patho_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#    ACCURACY += pd.read_table(FILE, header=0, index_col=0)

#for bat in range(1,17):#batch_rand):
#    for cv_batch in range(5):
#        FILE = home_dir + "/KEGG_1D_TCGA_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#        FILE = home_dir + "/unique_1D_KEGG_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#        FILE = home_dir + "/unique_1D_patho_result/conf/rand_" + str(bat) + "_" + str(cv_batch) + "_conftable.tsv"
#        ACCURACY += pd.read_table(FILE, header=0, index_col=0)




CLASS = n_categories

HIT = 0
Recall = np.zeros(CLASS)
Class_sum = np.zeros(CLASS)
Precision = np.zeros(CLASS)
Pred_sum = np.zeros(CLASS)
F1_score = np.zeros(CLASS)
Conf_matrix = np.zeros((CLASS,CLASS))
TP = np.zeros(CLASS)
TN = np.zeros(CLASS)
FP = np.zeros(CLASS)
FN = np.zeros(CLASS)
MATTHEW = np.zeros(CLASS)
for i in range(CLASS):
    Class_sum[i] = np.sum(ACCURACY.values[i,:])
    Pred_sum[i] = np.sum(ACCURACY.values[:,i])
    HIT += ACCURACY.values[i][i]
    TP[i] = ACCURACY.values[i][i]
    FN[i] = Class_sum[i] - ACCURACY.values[i][i]
    FP[i] = Pred_sum[i] - ACCURACY.values[i][i]
    Recall[i] = ACCURACY.values[i][i] /  Class_sum[i]
    Precision[i] = ACCURACY.values[i][i] / Pred_sum[i]
    F1_score[i] = 2 * Recall[i] * Precision[i] / (Recall[i] + Precision[i])
    Conf_matrix[i,:] = ACCURACY.values[i,:] / Class_sum[i]
SUM = np.sum(Class_sum)
TN = SUM - TP - FP - FN
ACC = HIT/SUM
RECALL = sum(Recall) / CLASS
PRECISION = sum(Precision) / CLASS
F1_SCORE = 2 * RECALL * PRECISION / (RECALL + PRECISION)
MATTHEW = Matthew(TP,TN,FP,FN)
MACRO_MATTHEW = np.sum(MATTHEW) / CLASS
trainLabel = pd.read_table(data_dir + '_result' + "/npy/label" + NORMAL + ".tsv", header=0)
label_List = np.asarray(trainLabel).reshape(-1,)
le = LabelEncoder()
le = le.fit(label_List)
label_List = le.transform(label_List)
labels = trainLabel["0"].unique()
labels.sort()
labels = labels
tmp = []
for i in range(len(labels)):
    tmp.append(labels[i].replace("TCGA-", ""))
labels = tmp

df_cmx = pd.DataFrame(Conf_matrix, index=labels, columns=labels)
plt.clf()
#plt.figure(figsize = (10,7))
#plt.title('Confusion matrix of 33 TCGA projects' + NORMAL)
plt.figure(figsize = (26,26))
plt.subplot(2,2,2)
plt.title('Confusion matrix of 46 cancer subtypes + Normal')
plt.show(sns.heatmap(df_cmx, annot=None, cmap="Blues", square=True))
#plt.savefig(result_dir + "/conf/heatmap_single_TCGA.png")
plt.savefig(result_dir + "/conf/heatmap_TCGA" + NORMAL + ".png")




In [ ]:
pd.set_option('display.max_columns', 50)
ACCURACY.to_csv(result_dir + "/conf/exsemble_total.tsv", sep='\t')



In [ ]:
# Grad-CAM visualisation
# For 281 genes

if not os.path.exists(result_dir + "/Grad_CAM"):
    os.mkdir(result_dir + "/Grad_CAM")

HilTable = pd.read_table(home_dir + "/Hilbert/Hilbert_256_128.tsv", header=0)
df_template = pd.read_table(home_dir + '/info/template.tsv', header=0)
df_pathway = pd.read_table(home_dir + '/KEGG/pathway.tsv', header=None)
df_unique = pd.read_table(home_dir + '/array_KEGG_1D/img_0.tsv', header=0)
df_image_temp = pd.read_table(home_dir + '/array_KEGG_256_128/img_0.tsv', header=0).values

tmp1 = df_template[df_template['KEGG_PATH'] != 0]
data_unique = tmp1.drop_duplicates()
data_No = tmp1['ENSG_no_var'].size
HilTable.columns = ['Num', 'X', 'Y']

GCAM = np.zeros((n_categories, 256, 128))
#GCAM = np.load(result_dir + "/Grad_CAM/Grad_CAM_batch_18.npy")


ind_list = np.empty((0,2), int)
important_gene = []
important_pathway = []
important_gene_id = []
important_pathway_id = []


NULL_list = []
for i in range(X_size):
    for j in range(Y_size):
        if(df_unique.iloc[i,j] == 60483):
            NULL_list.append([i,j])
# len(NULL_list); 526


trainLabel = pd.read_table(result_dir + "/npy/label.tsv", header=0)
label_List = np.asarray(trainLabel).reshape(-1,)
le = LabelEncoder()
le = le.fit(label_List)
label_List = le.transform(label_List)
tmp = trainLabel["0"].unique()
tmp.sort()
tmp1 = pd.DataFrame(tmp)

for bat in range(0, batch_rand):
    GCAM_batch = np.zeros((n_categories, X_size))
    df_unique = pd.read_table(home_dir + '/array_KEGG_1D/img_' + str(bat) + '.tsv', header=0).values
    image_list = np.load(result_dir + "/npy/image_" + str(bat) + ".npz")['arr_0'].astype(np.float32)
    image_list = np.reshape(image_list, (-1, 1, X_size))
    for cv_batch in range(0, n_Splits):
        test_index = np.load(result_dir + "/test_index/cv_batch_" + str(cv_batch) + "_index.npy")
        X_test = image_list[test_index]
        y_test = label_List[test_index]
        inputs = Input(shape=(1, X_size))
        x = Conv1D(32, 32, strides=1, activation='relu', padding='same', data_format="channels_first", name='block1_conv1', kernel_initializer='he_normal')(inputs)
        x = BatchNormalization(axis=1)(x,training=False)
        x = Conv1D(16, 128, strides=1, activation='relu', padding='same', data_format="channels_first", name='block1_conv2', kernel_initializer='he_normal')(x)
        x = BatchNormalization(axis=1)(x,training=False)
        x = Conv1D(2, 256, strides=1, activation='relu', padding='same', data_format="channels_first", name='block1_conv3', kernel_initializer='he_normal')(x)
        x = BatchNormalization(axis=1)(x,training=False)
        x = Flatten(name='flatten')(x)
        x = Dense(512, activation='relu', name='fc1')(x)
        x = Dropout(0.3, name='dropout1')(x,training=False)

        predictions = Dense(n_categories, activation='softmax', name='predictions')(x)
        VGG_model = Model(inputs=inputs, outputs=predictions)
        VGG_model.load_weights(result_dir + "/weight/rand_" + str(bat) + "_" + str(cv_batch) + "_model.h5")
        
        for i in range(n_categories):
            print("\r", bat, cv_batch, "class: ", tmp[i], end="")
            Images = X_test[y_test == i]
            Classes = y_test[y_test == i]
            Layer_name = 'block1_conv3'
            Input_model = VGG_model

            for k in range((Classes.size + batch_size - 1) // batch_size):
                start = k * batch_size
                end = min((k+1) * batch_size, Classes.size)
                GCAM_batch[i] += grad_batch(Input_model, Images[start:end], Classes[start:end], Layer_name, NULL_list, X_size, Y_size)
        K.clear_session()
        gc.collect()
        del VGG_model
        del x, inputs, predictions
        del X_test, y_test
    GCAM_temp = np.zeros((n_categories, 256, 128))
    for i in range(256):
        for j in range(128):
            if(df_image_temp[i,j] != 60483):
                tmp2 = HilTable[HilTable["X"]==i]
                tmp3 = tmp2[tmp2["Y"]==j]
                tmp4 = tmp3["Num"]
                tmp5 = math.ceil(tmp4 * data_No / 256 / 128)
                tmp6 = data_unique["KEGG_PATH"][tmp5]
                X_ = np.where(df_unique == df_image_temp[i,j])[0]
                for k in range(len(X_)):
                    if(len(X_)==1):
                        GCAM_temp[:,i,j] = GCAM_batch[:,X_].flatten()
                    else:
                        tmp11 = data_unique["KEGG_PATH"][X_[k]]
                        if(tmp11 == tmp6):
                            GCAM_temp[:,i,j] = GCAM_batch[:,X_[k]].flatten()
    GCAM += GCAM_temp
    del image_list
    np.save(result_dir + "/Grad_CAM/Grad_CAM_batch" + "_" + str(bat)+ ".npy", GCAM)

print("\r" + "ensembling")

for i in range(n_categories):
    picture = np.abs(GCAM[i]) / np.abs(GCAM[i]).max()
    picture2 = Grad_imaging(picture)
    picture2 = array_to_img(picture2)
    plt.imshow(picture2)
    plt.savefig(result_dir + "/Grad_CAM/Grad_CAM_gene_" + tmp[i] + ".png")

    gene_list = []
    gene_id_list = []   
    path_list = []
    path_id_list = []

    pathways = data_unique["KEGG_PATH"].drop_duplicates().values.tolist()
    path_index = 0
    picture3 = picture
    for j in range(200):
        print("\r", "category: ", i, "gene: ", j, end="")
        ind_list = np.unravel_index(np.argsort(np.abs(picture.flatten()))[::-1][j], picture.shape)
        gene_index = df_image_temp[ind_list[0]][ind_list[1]]
        gene_list.append(data_unique[data_unique["index"]==gene_index]["Gene_name"].drop_duplicates().values.tolist()[0])
        gene_id_list.append(data_unique[data_unique["index"]==gene_index]["ENSG_no_var"].drop_duplicates().values.tolist()[0])
    important_gene.append(sorted(list(set(gene_list)),key=gene_list.index))
    important_gene_id.append(sorted(list(set(gene_id_list)),key=gene_id_list.index))
    tmp5 = np.zeros(330)
    for j in range(330):
        print("\r", "category: ", i, "pathway: ", j, end="")
        tmp2 = data_unique[data_unique["KEGG_PATH"]==pathways[j]]
        tmp3 = math.floor(path_index / data_No * 256 * 128)
        tmp4 = math.floor((path_index + tmp2.shape[0]) / data_No * 256 * 128)
        for k in range(tmp3,tmp4):
            tmp5[j] += picture3[HilTable.iloc[k]["X"]][HilTable.iloc[k]["Y"]]
        tmp5[j] /= tmp2.shape[0]
        for k in range(tmp3,tmp4):
            picture3[HilTable.iloc[k]["X"]][HilTable.iloc[k]["Y"]] = tmp5[j]
        path_index += tmp2.shape[0]
    for k in range(10):
        path_list.append(df_pathway.iloc[np.abs(tmp5).argsort()[::-1][k]][1].replace(" - Homo sapiens (human)", ""))
        path_id_list.append(df_pathway.iloc[tmp5.argsort()[::-1][k]][0])
    picture3 = (picture3 - np.min(picture3)) / (np.max(picture3) - np.min(picture3))
    picture4 = Grad_imaging(picture3)
    picture4 = array_to_img(picture4)
    plt.imshow(picture4)
    plt.savefig(result_dir + "/Grad_CAM/Grad_CAM_pathway_" + tmp[i] + ".png")
    important_pathway.append(path_list)

pd.concat([tmp1, pd.DataFrame(important_gene)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_gene.tsv", index=False, sep='\t')
pd.concat([tmp1, pd.DataFrame(important_pathway)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_pathway.tsv", index=False, sep='\t')
pd.concat([tmp1, pd.DataFrame(important_gene_id)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_gene_id.tsv", index=False, sep='\t')
pd.concat([tmp1, pd.DataFrame(important_pathway_id)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_pathway_id.tsv", index=False, sep='\t')




In [ ]:
# Grad-CAM visualisation
# For 1D, 7,666 genes
batch_rand=32
NORMAL = ""
if not os.path.exists(result_dir + "/Grad_CAM"):
    os.mkdir(result_dir + "/Grad_CAM")

HilTable = pd.read_table(home_dir + "/Hilbert/Hilbert_256_128.tsv", header=0)
df_template = pd.read_table(home_dir + '/info/template.tsv', header=0)
df_pathway = pd.read_table(home_dir + '/KEGG/pathway.tsv', header=None)
df_unique = pd.read_table(home_dir + '/array_unique_1D/img_0.tsv', header=0)
df_image_temp = pd.read_table(home_dir + '/array_KEGG_256_128/img_0.tsv', header=0).values

tmp1 = df_template[df_template['KEGG_PATH'] != 0]
data_unique = tmp1.drop_duplicates()
data_No = tmp1['ENSG_no_var'].size
HilTable.columns = ['Num', 'X', 'Y']

NULL_list = []
for i in range(X_size):
    for j in range(Y_size):
        if(df_unique.iloc[i,j] == 60483):
            NULL_list.append([i,j])
# len(NULL_list); 526


trainLabel = pd.read_table(result_dir + "/npy/label" + NORMAL + ".tsv", header=0)
label_List = np.asarray(trainLabel).reshape(-1,)
le = LabelEncoder()
le = le.fit(label_List)
label_List = le.transform(label_List)
tmp = trainLabel["0"].unique()
tmp.sort()
tmp1 = pd.DataFrame(tmp)
for bat in range(0, batch_rand):
    GCAM = np.load(result_dir + "/Grad_CAM/Grad_CAM_batch_" + str(bat) + ".npy")
    ind_list = np.empty((0,2), int)
    important_gene = []
    important_gene_2 = []
    important_pathway = []
    important_gene_id = []
    important_pathway_id = []

    tmp5 = np.zeros((n_categories-1, 330))
    tmp6 = np.zeros((n_categories-1, 200))
    for i in range(n_categories-1):
        picture = np.abs(GCAM[i]) / np.abs(GCAM[i]).max()

        gene_list = []
        gene_list_2 = []
        gene_id_list = []   
        path_list = []
        path_id_list = []

        pathways = data_unique["KEGG_PATH"].drop_duplicates().values.tolist()
        for j in range(200):
            print("\r", "category: ", i, "gene: ", j, end="")
            ind_list = np.unravel_index(np.argsort(np.abs(picture.flatten()))[::-1][j], picture.shape)
            gene_index = df_image_temp[ind_list[0]][ind_list[1]]
            gene_list.append(data_unique[data_unique["index"]==gene_index]["Gene_name"].drop_duplicates().values.tolist()[0])
            gene_list_2.append(data_unique[data_unique["index"]==gene_index]["Gene_name"].values.tolist()[0])
        important_gene.append(sorted(list(set(gene_list)),key=gene_list.index))
        important_gene_2.append(sorted(list(set(gene_list_2)),key=gene_list.index))
        important_gene_2.append(np.sort(np.abs(picture.flatten()))[::-1][0:200])
    pd.concat([tmp1, pd.DataFrame(important_gene)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_gene" + NORMAL + "_" + str(bat)+ ".tsv", index=False, sep='\t')
    pd.concat([tmp1, pd.DataFrame(important_gene_2)], axis=1).to_csv(result_dir + "/Grad_CAM/Grad_CAM_gene_full" + NORMAL  + "_" + str(bat)+ ".tsv", index=False, sep='\t')




